# Model Selection
In diesem Notebook dient zum testen der Feature Engineering und Modellfunktionen. Außerdem wollen wir verschiedene Modelle ausprobieren, um uns für eine Modellart zu entscheiden.

In [ ]:
from src.features.build_features import feature_engineering
from src.data import load_full_data
import mord
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from src.models import nn_ordinal_labels
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from src.models.predict_model import ordinal_predict
import lightgbm as lgb

In [2]:
data = load_full_data()
data

,index,scene_id,duration,rating,sessionID,ageGroup,bicycleUse,bikeReasonsVar,district,gender,...,FS-Geschwindigkeit,RVA-Lage,RVA-Oberfläche,Tr_li-Markierung,Tr_li-baulTrennung,Tr_re-Markierung,Parken,besondere Merkmale,FS-Breite,Verkehrsaufkommen
0,1,01_CP_C_1226,2835.0,3.0,6bbf3fed-46ab-4221-930b-4ac3fa0acff8,4.0,0.0,NaN,Neukölln,m,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,189,01_CP_C_1226,3701.0,3.0,dc6c5279-3294-4d41-bdef-00b7bdf4ae3d,5.0,1.0,NaN,Treptow-Köpenick,m,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,01_CP_C_1226,3381.0,3.0,18b5b5dd-9369-4dcd-beae-34efe63d44ad,1.0,1.0,NaN,Charlottenburg-Wilmersdorf,m,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,01_CP_C_1226,1784.0,3.0,05a41c9d-ecd5-48ea-a1b3-71598f8e066f,3.0,1.0,NaN,Charlottenburg-Wilmersdorf,w,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,01_CP_C_1226,6312.0,2.0,b398c275-6ffe-491a-b087-3d95982e494f,4.0,NaN,NaN,Tempelhof-Schöneberg,m,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307707,13,01_SE_A_68,4317.0,1.0,d835cac8-a54b-4e22-a1bf-b2390d3fd1c8,5.0,1.0,NaN,Tempelhof-Schöneberg,m,...,NaN,NaN,NaN,NaN,NaN,NaN,nein,Spielstraße,schmal,normal
307708,14,01_SE_A_68,2700.0,1.0,a90a5517-727d-4fcd-a8ac-75eaa44e0bcc,5.0,1.0,NaN,Mitte,m,...,NaN,NaN,NaN,NaN,NaN,NaN,nein,Spielstraße,schmal,normal
307709,12,01_SE_A_68,4150.0,1.0,897d4978-cce1-4422-a058-31be453411fb,5.0,0.0,NaN,Tempelhof-Schöneberg,m,...,NaN,NaN,NaN,NaN,NaN,NaN,nein,Spielstraße,schmal,normal
307710,11,01_SE_A_68,2252.0,0.0,cb90c5fa-e251-45dc-a27c-54cf2af6ac0c,2.0,1.0,NaN,Tempelhof-Schöneberg,w,...,NaN,NaN,NaN,NaN,NaN,NaN,nein,Spielstraße,schmal,normal


Da sich die Merkmale für die einzelnen Szenarienarten (Hauptverkehrsstraßen, Nebenverkehrsstraßen und RVA im Seitenraum) stark unterscheiden, sollten für jede Szenarienart ein eigenes Modell erstellt werden. Die Wahl der Modellart, sollte aber für alle gemeinschaftlich gewählt werden.

Für das Feature Engineering werden folgende Strategien angewendet:
- Es werden Spalten entfernt, die größtenteils fehlende Werte für alle Teilnehmer besitzen. (#TODO where to find)
- Es werden Spalten entfernt, deren Informationen in andern Spalten zu finden sind.
- Es werden für das Modell unnütze Spalten entfernt.
- Fehlende Werte werden durch die Mittelwerte der jeweiligen Spalte ersetzt.
- index hat nun einen Maximalwert von 11
- Numerische Werte werden normalisiert. (außer "rating")
- RVA-Breite und Tr_li-Breite werden als numerische Werte behandelt.
- Alle anderen Variablen werden One-Hot transformiert.

In [3]:
experiments =["MS", "CP", "SE"]

datasets = {}

for ex in experiments:
    datasets[ex] = feature_engineering(data[(data["Experiment"] == ex) 
                                            & (data["Kamera"] == "C")
                                           ].drop(["Experiment",
                                                                             "Kamera"],
                                                                             axis=1).dropna(axis=1,
                                                                                            how="all"))

Für eine erste Einschätzung der Modellqualität, setzen wir für schnellere Iterationszeiten auf einen simplen Train/Test split. Im finalen Modelltraining, werden wir dann auf K-Fold Cross Validation setzen.

In [4]:
train_datasets = {}
test_datasets = {}
split_value = 0.9

for dataset in datasets.keys():
    current_dataset = datasets[dataset]
    split = int(current_dataset.shape[0]*split_value)
    train = current_dataset.iloc[:split]
    test = current_dataset.iloc[split:]
    train_datasets[dataset] = (train.drop(["rating"], axis=1), train["rating"])
    test_datasets[dataset] = (test.drop(["rating"], axis=1), test["rating"])
    print("Dataset: ", dataset,
          " - Original size:", current_dataset.shape[0],
          " - Train size:", train_datasets[dataset][0].shape[0],
          " - Test size:", test_datasets[dataset][0].shape[0])

Dataset:  MS  - Original size: 55604  - Train size: 50043  - Test size: 5561
Dataset:  CP  - Original size: 86825  - Train size: 78142  - Test size: 8683
Dataset:  SE  - Original size: 31560  - Train size: 28404  - Test size: 3156


# Ordinale Regression
Da wir mit einem Datensatz basierend auf Likert-Skala Daten arbeiten, bietet die Ordinale Regression die beste theoretische Basis.

Quelle:
J. D. M. Rennie and N. Srebro, “Loss Functions for Preference Levels : Regression with Discrete Ordered Labels,” in Proceedings of the IJCAI Multidisciplinary Workshop on Advances in Preference Handling, 2005.

In [5]:
for dataset in train_datasets.keys():
    print("Working on:", dataset)
    x_train = train_datasets[dataset][0]
    x_test = test_datasets[dataset][0]
    y_train = train_datasets[dataset][1].astype('int32')
    y_test = test_datasets[dataset][1].astype('int32')
    print("Building Model")
    mul_lr = mord.LogisticAT(**{"alpha": 0.7939428335577381, "max_iter": 199}).fit(x_train, y_train)
    y_pred = mul_lr.predict(x_test)
    print("Mean Absolute Error on Test Set:", mean_absolute_error(y_test, y_pred))

Working on: MS
Building Model
Mean Absolute Error on Test Set: 0.5641071749685308
Working on: CP
Building Model
Mean Absolute Error on Test Set: 0.565127260163538
Working on: SE
Building Model
Mean Absolute Error on Test Set: 0.6204055766793409


# Deep Neural Network
Da wir sehr viele einzelne Datenpunkte haben und viele kategorische Merkmale, welche wir nicht gut numerisch darstellen können, könnte sich der Datensatz gut für ein Neuronales Netz eignen. Hier könnte das NN theoretisch ein eigenes Embedding für die kategorischen One-Hot Merkmale lernen und außerdem Effekte zwischen verschieden Merkmalen abbilden.

Um Likert Skala Daten mit dem NN zu trainieren, nutzen wir einen Trick der häufig für verschiedene Modellarten benutzt wird um die ordinale Eigenschaft der Daten abzubilden.
Genaueres dazu im NN Kontext unter:
Cheng, Jianlin, Zheng Wang, and Gianluca Pollastri. "A neural network approach to ordinal regression." 2008 IEEE International Joint Conference on Neural Networks (IEEE World Congress on Computational Intelligence). IEEE, 2008.

In [6]:
def create_model():
    print("Building Model")
    model = []
    model.append(Dense(x_train.shape[0], activation="relu"))
    model.append(Dropout(0.3))
    model.append(Dense(128, activation="relu"))
    model.append(Dropout(0.3))
    model.append(Dense(10, activation="relu"))
    model.append(Dense(3, activation="sigmoid"))

    model = tf.keras.models.Sequential(model)
    model.compile(loss=MeanSquaredError(),
                    optimizer=Adam())
    return model


for dataset in train_datasets.keys():
    print("Working on:", dataset)
    x_train, y_train = train_datasets[dataset]
    y_train = nn_ordinal_labels(y_train)
    x_test, y_test = test_datasets[dataset]
    y_test_val = nn_ordinal_labels(y_test)  
    early_stop = EarlyStopping(monitor='val_loss',
                                  min_delta=0,
                                  patience=1, mode='auto')

    my_model = KerasRegressor(build_fn=create_model,
                              epochs=10,
                              batch_size=512,
                              callbacks=[early_stop],
                              validation_data=(x_test.to_numpy(), y_test_val),
                              verbose=0)
    print("Training Model")
    my_model.fit(x_train.to_numpy(), y_train)
    y_pred = my_model.predict(x_test.to_numpy())
    y_pred = ordinal_predict(y_pred)
    print("Mean Absolute Error on Test Set:", mean_absolute_error(y_test, y_pred))

Working on: MS
Training Model
Building Model
Mean Absolute Error on Test Set: 0.5630282323323144
Working on: CP
Training Model
Building Model
Mean Absolute Error on Test Set: 0.4932626972244616
Working on: SE
Training Model
Building Model
Mean Absolute Error on Test Set: 0.6108998732572877


# Boosting Decision Trees
Boosting ist eine populäre Methode die gute Ergebnisse erreichen können.

In [7]:
for dataset in train_datasets.keys():
    print("Working on:", dataset)
    x_train, y_train = train_datasets[dataset]
    x_test, y_test = test_datasets[dataset]
    model = lgb.LGBMRegressor(n_estimators=500)
    print("Fitting Model")
    model.fit(x_train.to_numpy(), y_train.astype(int), eval_set=(x_test.to_numpy(), y_test.astype(int)), verbose=False)
    y_pred = model.predict(x_test)
    print("Absolute Error on Test Set:", mean_absolute_error(y_test.values, y_pred))

Working on: MS
Fitting Model
Absolute Error on Test Set: 0.619153329018099
Working on: CP
Fitting Model
Absolute Error on Test Set: 0.5349916929776412
Working on: SE
Fitting Model
Absolute Error on Test Set: 0.6207815551506587


Die Ordinale Regression und das Deep Neural Network bieten allgemein die besten Ergebnisse. Aufgrund der kürzeren Iterationszeiten der Regression, werden wir das NN nicht weiter verfolgen. Dies könnte jedoch eine spannende zukünftige Erweiterung sein.